In [12]:
import os
import datetime
from datetime import timedelta
import quandl
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import plotnine as p9
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore
import pandas_market_calendars as mcal


import warnings
import functools
import wrds
import sys
# Ignore all warnings
warnings.filterwarnings("ignore")
from scipy.stats import linregress


In [56]:
db = wrds.Connection()


OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [ ]:
earnings_dates_for_tickers = pd.read_csv("results.csv")
earnings_dates_for_tickers

,Unnamed: 0.1,Unnamed: 0,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date,trade_entry,...,straddle_entry_strike,IV_25c,IV_25p,IV_10c,IV_10p,IV_50c,skew_25,skew_25_decile,kurt_25,kurt_25_decile
0,0,0,396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,2018-02-15 00:00:00+00:00,2018-02-14,...,70000.0,NaN,0.594952,NaN,0.593143,0.577625,NaN,NaN,NaN,NaN
1,1,1,1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,2018-05-15 00:00:00+00:00,2018-05-14,...,NaN,0.532359,NaN,NaN,0.625772,0.570963,NaN,NaN,NaN,NaN
2,2,2,2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,2018-08-15 00:00:00+00:00,2018-08-14,...,67500.0,0.841737,0.995516,0.956398,0.948521,0.948011,-0.721488,"(-0.762, -0.586]",-1.635179,"(-1.869, -1.159]"
3,3,3,2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,2018-11-20 00:00:00+00:00,2018-11-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,2019-02-21 00:00:00+00:00,2019-02-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19471,19471,19471,15969,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER,2023-02-13 00:00:00+00:00,2023-02-13,...,NaN,NaN,NaN,0.516541,0.689797,0.461846,NaN,NaN,NaN,NaN
19472,19472,19472,16912,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER,2023-05-03 00:00:00+00:00,2023-05-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19473,19473,19473,17936,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER,2023-08-07 00:00:00+00:00,2023-08-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19474,19474,19474,18652,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER,2023-11-01 00:00:00+00:00,2023-11-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
csv_file_path = "secids.csv"

# Reading the CSV file into a NumPy array
secids = np.genfromtxt(csv_file_path, delimiter=',')
secids

array([101149., 205456., 210354., ..., 211262., 207854., 105959.])

In [ ]:
options_dict = {} 
for year in range(2017, 2024):
    print(year)
    table_name = f"optionm.opprcd{year}"  # Generate table name dynamically
    query = f"""
    SELECT
        date, secid, symbol, cp_flag, expiry_indicator, volume, strike_price, exdate, open_interest, impl_volatility, best_bid, best_offer, delta, gamma, vega, theta
    FROM 
        {table_name} a 
    WHERE
        a.secid IN ({','.join(map(str, secids))}) AND
        a.volume > 0 AND
        a.exdate - a.date <= 38
    """
    options_dict[year] = db.raw_sql(query, date_cols=['date'])

2017
2018
2019
2020
2021
2022
2023


In [ ]:
options_dict[2017].to_parquet("Options2017.parquet")

In [ ]:
for year in range(2018,2024):
    options_dict[year].to_parquet("Options"+str(year)+".parquet")

In [ ]:
options_dict

{}

In [ ]:
for year in range(2017, 2024):
    # Get the DataFrame for the current year
    options = options_dict[year]
    
    # Split the 'symbol' column and extract the first substring as 'ticker'
    options['ticker'] = options['symbol'].str.split(' ').str[0]
    
    # Set the multi-index using 'date' and 'ticker'
    options.set_index(['ticker','date','exdate'], inplace=True)
    
    # Update the options_dict with the modified DataFrame
    options_dict[year] = options

In [ ]:
stock_prices = pd.read_parquet("stock_price_ED_20182023.parquet")
stock_prices

,date,permno,prc,bid,ask,ticker,comnam,permco
0,2018-02-07,87432,68.06,68.06000,68.07000,A,AGILENT TECHNOLOGIES INC,36364.0
1,2018-02-08,87432,65.05,65.03000,65.05000,A,AGILENT TECHNOLOGIES INC,36364.0
2,2018-02-09,87432,66.98,66.97000,66.98000,A,AGILENT TECHNOLOGIES INC,36364.0
3,2018-02-12,87432,68.43,68.40000,68.41000,A,AGILENT TECHNOLOGIES INC,36364.0
4,2018-02-13,87432,68.34,68.33000,68.34000,A,AGILENT TECHNOLOGIES INC,36364.0
...,...,...,...,...,...,...,...,...
147109,2023-11-03,13788,162.23,162.19000,162.23000,ZTS,ZOETIS INC,54327.0
147110,2023-11-06,13788,163.17,163.20000,163.27000,ZTS,ZOETIS INC,54327.0
147111,2023-11-07,13788,167.16,167.24001,167.28000,ZTS,ZOETIS INC,54327.0
147112,2023-11-08,13788,170.87,170.89000,170.92999,ZTS,ZOETIS INC,54327.0


In [ ]:
earnings_dates_for_tickers

,Unnamed: 0.1,Unnamed: 0,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date,trade_entry,...,straddle_entry_strike,IV_25c,IV_25p,IV_10c,IV_10p,IV_50c,skew_25,skew_25_decile,kurt_25,kurt_25_decile
0,0,0,396,A,Agilent Technologies Inc,A US,2018-02-14 16:05:00,ER,2018-02-15 00:00:00+00:00,2018-02-14,...,70000.0,NaN,0.594952,NaN,0.593143,0.577625,NaN,NaN,NaN,NaN
1,1,1,1298,A,Agilent Technologies Inc,A US,2018-05-14 16:05:00,ER,2018-05-15 00:00:00+00:00,2018-05-14,...,NaN,0.532359,NaN,NaN,0.625772,0.570963,NaN,NaN,NaN,NaN
2,2,2,2023,A,Agilent Technologies Inc,A US,2018-08-14 16:05:00,ER,2018-08-15 00:00:00+00:00,2018-08-14,...,67500.0,0.841737,0.995516,0.956398,0.948521,0.948011,-0.721488,"(-0.762, -0.586]",-1.635179,"(-1.869, -1.159]"
3,3,3,2754,A,Agilent Technologies Inc,A US,2018-11-19 16:05:00,ER,2018-11-20 00:00:00+00:00,2018-11-19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,3297,A,Agilent Technologies Inc,A US,2019-02-20 16:05:00,ER,2019-02-21 00:00:00+00:00,2019-02-20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19471,19471,19471,15969,ZTS,Zoetis Inc,ZTS US,2023-02-14 07:00:00,ER,2023-02-13 00:00:00+00:00,2023-02-13,...,NaN,NaN,NaN,0.516541,0.689797,0.461846,NaN,NaN,NaN,NaN
19472,19472,19472,16912,ZTS,Zoetis Inc,ZTS US,2023-05-04 07:00:00,ER,2023-05-03 00:00:00+00:00,2023-05-03,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19473,19473,19473,17936,ZTS,Zoetis Inc,ZTS US,2023-08-08 07:00:00,ER,2023-08-07 00:00:00+00:00,2023-08-07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19474,19474,19474,18652,ZTS,Zoetis Inc,ZTS US,2023-11-02 07:00:00,ER,2023-11-01 00:00:00+00:00,2023-11-01,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
earnings_dates_for_tickers['trade_entry'] = pd.to_datetime(earnings_dates_for_tickers['trade_entry'])

# Extract only the date part (YYYY-MM-DD)
earnings_dates_for_tickers['trade_entry'] = earnings_dates_for_tickers['trade_entry'].dt.date
earnings_dates_for_tickers['trade_entry']

0        2018-02-14
1        2018-05-14
2        2018-08-14
3        2018-11-19
4        2019-02-20
            ...    
19471    2023-02-13
19472    2023-05-03
19473    2023-08-07
19474    2023-11-01
19475    2024-02-12
Name: trade_entry, Length: 19476, dtype: object

In [ ]:
earnings_dates_for_tickers['trade_entry'] = pd.to_datetime(earnings_dates_for_tickers['trade_entry'])
earnings_dates_for_tickers['trade_entry']


0       2018-02-14
1       2018-05-14
2       2018-08-14
3       2018-11-19
4       2019-02-20
           ...    
19471   2023-02-13
19472   2023-05-03
19473   2023-08-07
19474   2023-11-01
19475   2024-02-12
Name: trade_entry, Length: 19476, dtype: datetime64[ns]

In [ ]:
earnings_dates_for_tickers['trade_exit'] = pd.to_datetime(earnings_dates_for_tickers['trade_exit'])

# Extract only the date part (YYYY-MM-DD)
earnings_dates_for_tickers['trade_exit'] = earnings_dates_for_tickers['trade_exit'].dt.date
earnings_dates_for_tickers['trade_exit'] = pd.to_datetime(earnings_dates_for_tickers['trade_exit'])
earnings_dates_for_tickers['trade_exit']



0       2018-02-15
1       2018-05-15
2       2018-08-15
3       2018-11-20
4       2019-02-21
           ...    
19471   2023-02-14
19472   2023-05-04
19473   2023-08-08
19474   2023-11-02
19475   2024-02-13
Name: trade_exit, Length: 19476, dtype: datetime64[ns]

In [ ]:
stock_prices.set_index(['date','ticker'],inplace = True)
stock_prices

,,permno,prc,bid,ask,comnam,permco
date,ticker,,,,,,
2018-02-07,A,87432,68.06,68.06000,68.07000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-08,A,87432,65.05,65.03000,65.05000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-09,A,87432,66.98,66.97000,66.98000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-12,A,87432,68.43,68.40000,68.41000,AGILENT TECHNOLOGIES INC,36364.0
2018-02-13,A,87432,68.34,68.33000,68.34000,AGILENT TECHNOLOGIES INC,36364.0
...,...,...,...,...,...,...,...
2023-11-03,ZTS,13788,162.23,162.19000,162.23000,ZOETIS INC,54327.0
2023-11-06,ZTS,13788,163.17,163.20000,163.27000,ZOETIS INC,54327.0
2023-11-07,ZTS,13788,167.16,167.24001,167.28000,ZOETIS INC,54327.0


In [ ]:
def entry_stock_price(row):
    try:
        # Check if there is an index match in stock_prices
        if (row['trade_entry'], row['Short_ticker']) in stock_prices.index:
            # Get the index position of the match
            match_index = stock_prices.index.get_loc((row['trade_entry'], row['Short_ticker']))
            # Get the previous 5 rows using slicing
            previous_rows = stock_prices.iloc[match_index-5:match_index+1]

            # get the 'prc' of the rows and return all of them as a list
            return previous_rows['prc'].to_list()
            
        else:
            return np.nan*6
    except KeyError:
        return np.nan*6

earnings_dates_for_tickers[['prc_5Dbefore', 'prc_4Dbefore', 'prc_3Dbefore', 'prc_2Dbefore','prc_1Dbefore','stock_price_entry']] = earnings_dates_for_tickers.apply(entry_stock_price, axis=1, result_type = 'expand')

In [ ]:
def entry_stock_price(row):
    try:
        return stock_prices.loc[(row['trade_entry'], row['Short_ticker'])]['prc']
    except KeyError:
        return np.nan
earnings_dates_for_tickers['stock_price_entry'] = earnings_dates_for_tickers.apply(entry_stock_price,axis = 1)

In [ ]:
def exit_stock_price(row):
    try:
        return stock_prices.loc[(row['trade_exit'], row['Short_ticker'])]['prc']
    except KeyError:
        return np.nan
earnings_dates_for_tickers['stock_price_exit'] = earnings_dates_for_tickers.apply(exit_stock_price,axis = 1)

In [ ]:
earnings_dates_for_tickers['year_entry'] = earnings_dates_for_tickers['trade_entry'].dt.year

# Create 'year_exit' column containing the year of 'trade_exit'
earnings_dates_for_tickers['year_exit'] = earnings_dates_for_tickers['trade_exit'].dt.year
earnings_dates_for_tickers['year_entry']

0        2018
1        2018
2        2018
3        2018
4        2019
         ... 
19471    2023
19472    2023
19473    2023
19474    2023
19475    2024
Name: year_entry, Length: 19476, dtype: int32

In [ ]:
approximation_error = abs(1000 * earnings_dates_for_tickers['stock_price_entry'] - 1000 * earnings_dates_for_tickers['stock_price_entry'].round())
approximation_error.mean()

245.41648013548803

In [ ]:
earnings_dates_for_tickers['Unnamed: 0']

0            0
1            1
2            2
3            3
4            4
         ...  
19471    19471
19472    19472
19473    19473
19474    19474
19475    19475
Name: Unnamed: 0, Length: 19476, dtype: int64

In [120]:
def straddle_iv_time(row):
    if row['Unnamed: 0'] % 100 == 0:
        print(row['Unnamed: 0'])
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return [np.nan]*24
    try:

        IV_1 = []
        time_1 = []
        IV_2 = []
        time_2 = []

        # Trading Entry Date

        options = options_dict[row['year_entry']].loc[(row['Short_ticker'], row['trade_entry'])]

        # if there is no two exdates, add nan to the list
        exdates = options.index.get_level_values('exdate').unique().sort_values()
        if len(exdates) < 2:
            return [np.nan]*24
        else:
            cloest_two_exdates = exdates[:2]
            # get the option with the closet exdate
            options_1 = options.loc[cloest_two_exdates[0]]
            options_1['abs_difference'] = abs(options_1['strike_price'] - row['stock_price_entry']*1000)

            # Filter the DataFrame for r
            # ows with the minimum absolute difference
            min_abs_difference = options_1['abs_difference'].min()
            filtered_options = options_1[options_1['abs_difference'] == min_abs_difference]

            if len(filtered_options) <=2:
                b = True
                IV_1.append(filtered_options['impl_volatility'].mean())
                
                # calculate the calendar days between 'trade_entry' and closest_two_exdates[0]
                time_1.append((pd.to_datetime(cloest_two_exdates[0]) - row['trade_entry']).days/365.0)
            else:
                IV_1.append(np.nan)
                time_1.append(np.nan)

            # second closest exdate
            options_2 = options.loc[cloest_two_exdates[1]]
            options_2['abs_difference'] = abs(options_2['strike_price'] - row['stock_price_entry']*1000)

            # Filter the DataFrame for rows with the minimum absolute difference
            min_abs_difference = options_2['abs_difference'].min()
            filtered_options = options_2[options_2['abs_difference'] == min_abs_difference]

            if len(filtered_options) <=2:
                IV_2.append(filtered_options['impl_volatility'].mean())
                time_2.append((pd.to_datetime(cloest_two_exdates[1]) - row['trade_entry']).days/365.0)
            else:
                IV_2.append(np.nan)
                time_2.append(np.nan)


        # 1-5 days before earnings

            nyse = mcal.get_calendar('NYSE')
            current_date = row['trade_entry']
            schedule = nyse.schedule(start_date=current_date - pd.DateOffset(days=20), end_date=current_date)
            for i in range(1, 6):
                previous_trading_day = schedule.iloc[-(i+1)].name
                previous_year = previous_trading_day.year
                try:
                    options = options_dict[previous_year].loc[( row['Short_ticker'], previous_trading_day)]
                    # get the option with the closet exdate
                    options_1 = options.loc[cloest_two_exdates[0]]
                    options_1['abs_difference'] = abs(options_1['strike_price'] - row[f'prc_{i}Dbefore']*1000)

                        # Filter the DataFrame for rows with the minimum absolute difference
                    min_abs_difference = options_1['abs_difference'].min()
                    filtered_options = options_1[options_1['abs_difference'] == min_abs_difference]

                    if len(filtered_options) <= 2:
                        IV_1.append(filtered_options['impl_volatility'].mean())
                            
                            # calculate the calendar days between 'trade_entry' and closest_two_exdates[0]
                        time_1.append((pd.to_datetime(cloest_two_exdates[0]) - previous_trading_day).days/365.0)
                    else:
                        IV_1.append(np.nan)
                        time_1.append(np.nan)

                        # second closest exdate
                    options_2 = options.loc[cloest_two_exdates[1]]
                    options_2['abs_difference'] = abs(options_2['strike_price'] - row[f'prc_{i}Dbefore' ]*1000)

                        # Filter the DataFrame for rows with the minimum absolute difference
                    min_abs_difference = options_2['abs_difference'].min()
                    filtered_options = options_2[options_2['abs_difference'] == min_abs_difference]

                    if len(filtered_options) <=2:
                        IV_2.append(filtered_options['impl_volatility'].mean())
                        time_2.append((pd.to_datetime(cloest_two_exdates[1]) - previous_trading_day).days/365.0)
                    else:
                        IV_2.append(np.nan)
                        time_2.append(np.nan)
                except:
                    if len(IV_1) == i:
                        IV_1.append(np.nan)
                    if len(time_1) == i:
                        time_1.append(np.nan)
                    if len(IV_2) == i:
                        IV_2.append(np.nan)
                    if len(time_2) == i:
                        time_2.append(np.nan)

            # if there is no two exdates, add nan to the list
            


        # return IV and time
        return IV_1 + time_1 + IV_2 + time_2

    except:
        return [np.nan]*24




earnings_dates_for_tickers[['straddle_iv_T1', 'straddle_iv_1d_T1','straddle_iv_2d_T1','straddle_iv_3d_T1','straddle_iv_4d_T1','straddle_iv_5d_T1',
                            'time_T1', 'time_1d_T1','time_2d_T1','time_3d_T1','time_4d_T1','time_5d_T1',
                            'straddle_iv_T2', 'straddle_iv_1d_T2','straddle_iv_2d_T2','straddle_iv_3d_T2','straddle_iv_4d_T2','straddle_iv_5d_T2',
                            'time_T2', 'time_1d_T2','time_2d_T2','time_3d_T2','time_4d_T2','time_5d_T2']] = earnings_dates_for_tickers.apply(straddle_iv_time, axis = 1, result_type = 'expand')

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [121]:
earnings_dates_for_tickers.to_csv("Results_0510.csv")

In [78]:
cols_iv_time = ['straddle_iv_T1', 'straddle_iv_1d_T1','straddle_iv_2d_T1','straddle_iv_3d_T1','straddle_iv_4d_T1','straddle_iv_5d_T1',
                            'time_T1', 'time_1d_T1','time_2d_T1','time_3d_T1','time_4d_T1','time_5d_T1',
                            'straddle_iv_T2', 'straddle_iv_1d_T2','straddle_iv_2d_T2','straddle_iv_3d_T2','straddle_iv_4d_T2','straddle_iv_5d_T2',
                            'time_T2', 'time_1d_T2','time_2d_T2','time_3d_T2','time_4d_T2','time_5d_T2']
#hundred.drop(columns = cols_iv_time,axis = 1, inplace = True)
earnings_dates_for_tickers.drop(columns = cols_iv_time,axis = 1,inplace = True)

In [115]:

def straddle_iv_time_v1(row):
    if row['Unnamed: 0'] % 100 == 0:
        print(row['Unnamed: 0'])
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return [np.nan]*24
    try:
        IV_1 = []
        time_1 = []
        IV_2 = []
        time_2 = []

        # Trading Entry Date
        options = options_dict[row['year_entry']].loc[(row['Short_ticker'],row['trade_entry'])]
        cloest_two_exdates = options.index.get_level_values('exdate').unique().sort_values()[:2]

        # get the option with the closet exdate
        options_1 = options.loc[cloest_two_exdates[0]]
        options_1['abs_difference'] = abs(options_1['strike_price'] - row['stock_price_entry']*1000)

        # Filter the DataFrame for rows with the minimum absolute difference
        min_abs_difference = options_1['abs_difference'].min()
        filtered_options = options_1[options_1['abs_difference'] == min_abs_difference]

        if len(filtered_options) <=2:
            IV_1.append(filtered_options['impl_volatility'].mean())
            
            # calculate the calendar days between 'trade_entry' and closest_two_exdates[0]
            time_1.append((pd.to_datetime(cloest_two_exdates[0]) - row['trade_entry']).days/365.0)
        else:
            IV_1.append(np.nan)
            time_1.append(np.nan)

        # second closest exdate
        options_2 = options.loc[cloest_two_exdates[1]]
        options_2['abs_difference'] = abs(options_2['strike_price'] - row['stock_price_entry']*1000)

        # Filter the DataFrame for rows with the minimum absolute difference
        min_abs_difference = options_2['abs_difference'].min()
        filtered_options = options_2[options_2['abs_difference'] == min_abs_difference]

        if len(filtered_options) <=2:
            IV_2.append(filtered_options['impl_volatility'].mean())
            time_2.append((pd.to_datetime(cloest_two_exdates[1]) - row['trade_entry']).days/365.0)
        else:
            IV_2.append(np.nan)
            time_2.append(np.nan)


        # 1-5 days before earnings

        nyse = mcal.get_calendar('NYSE')
        current_date = row['trade_entry']
        schedule = nyse.schedule(start_date=current_date - pd.DateOffset(days=10), end_date=current_date)

        for i in range(1, 6):
            previous_trading_day = schedule.iloc[-(i+1)].name
            previous_year = previous_trading_day.year
            options = options_dict[previous_year].loc[(row['Short_ticker'],previous_trading_day)]
            # get the option with the closet exdate
            options_1 = options.loc[cloest_two_exdates[0]]
            options_1['abs_difference'] = abs(options_1['strike_price'] - row[f'prc_{i}Dbefore']*1000)

            # Filter the DataFrame for rows with the minimum absolute difference
            min_abs_difference = options_1['abs_difference'].min()
            filtered_options = options_1[options_1['abs_difference'] == min_abs_difference]

            if len(filtered_options) <=2:
                IV_1.append(filtered_options['impl_volatility'].mean())
                
                # calculate the calendar days between 'trade_entry' and closest_two_exdates[0]
                time_1.append((pd.to_datetime(cloest_two_exdates[0]) - previous_trading_day).days/365.0)
            else:
                IV_1.append(np.nan)
                time_1.append(np.nan)

            # second closest exdate
            options_2 = options.loc[cloest_two_exdates[1]]
            options_2['abs_difference'] = abs(options_2['strike_price'] - row[f'prc_{i}Dbefore' ]*1000)

            # Filter the DataFrame for rows with the minimum absolute difference
            min_abs_difference = options_2['abs_difference'].min()
            filtered_options = options_2[options_2['abs_difference'] == min_abs_difference]

            if len(filtered_options) <=2:
                IV_2.append(filtered_options['impl_volatility'].mean())
                time_2.append((pd.to_datetime(cloest_two_exdates[1]) - previous_trading_day).days/365.0)
            else:
                IV_2.append(np.nan)
                time_2.append(np.nan)



        # return IV and time
        return IV_1 + time_1 + IV_2 + time_2

    except:
        return [np.nan]*24

hundred[['straddle_iv_T1', 'straddle_iv_1d_T1','straddle_iv_2d_T1','straddle_iv_3d_T1','straddle_iv_4d_T1','straddle_iv_5d_T1',
                            'time_T1', 'time_1d_T1','time_2d_T1','time_3d_T1','time_4d_T1','time_5d_T1',
                            'straddle_iv_T2', 'straddle_iv_1d_T2','straddle_iv_2d_T2','straddle_iv_3d_T2','straddle_iv_4d_T2','straddle_iv_5d_T2',
                            'time_T2', 'time_1d_T2','time_2d_T2','time_3d_T2','time_4d_T2','time_5d_T2']] = hundred.apply(straddle_iv_time_v1, axis = 1, result_type = 'expand')

0


In [28]:
earnings_dates_for_tickers = pd.read_csv('Results_0510.csv')

In [29]:
forward_vol_0d = np.sqrt((earnings_dates_for_tickers['time_T2']*earnings_dates_for_tickers['straddle_iv_T2']**2.0-earnings_dates_for_tickers['time_T1']*earnings_dates_for_tickers['straddle_iv_T1']**2.0)/(earnings_dates_for_tickers['time_T2']-earnings_dates_for_tickers['time_T1']))
event_var_0d = earnings_dates_for_tickers['time_T1']*(earnings_dates_for_tickers['straddle_iv_T1']**2.0-forward_vol_0d**2.0)
event_var_0d

0        0.001523
1        0.003208
2        0.006852
3             NaN
4             NaN
           ...   
19471    0.001283
19472         NaN
19473         NaN
19474         NaN
19475         NaN
Length: 19476, dtype: float64

In [30]:
((earnings_dates_for_tickers['time_5d_T2']-earnings_dates_for_tickers['time_5d_T1'])-(earnings_dates_for_tickers['time_T2']-earnings_dates_for_tickers['time_T1'])).min()

-1.1102230246251565e-16

In [31]:
np.nanmean(np.sqrt(event_var_0d))

0.06829962706721962

In [32]:
forward_vol_5d = np.sqrt((earnings_dates_for_tickers['time_5d_T2']*earnings_dates_for_tickers['straddle_iv_5d_T2']**2.0-earnings_dates_for_tickers['time_5d_T1']*earnings_dates_for_tickers['straddle_iv_5d_T1']**2.0)/(earnings_dates_for_tickers['time_5d_T2']-earnings_dates_for_tickers['time_5d_T1']))
event_var_5d = earnings_dates_for_tickers['time_5d_T1']*(earnings_dates_for_tickers['straddle_iv_5d_T1']**2.0-forward_vol_5d**2.0)
np.nanmean(np.sqrt(event_var_5d))

0.07052639536051296

In [33]:
forward_vol_4d = np.sqrt((earnings_dates_for_tickers['time_4d_T2']*earnings_dates_for_tickers['straddle_iv_4d_T2']**2.0-earnings_dates_for_tickers['time_4d_T1']*earnings_dates_for_tickers['straddle_iv_4d_T1']**2.0)/(earnings_dates_for_tickers['time_4d_T2']-earnings_dates_for_tickers['time_4d_T1']))
event_var_4d = earnings_dates_for_tickers['time_4d_T1']*(earnings_dates_for_tickers['straddle_iv_4d_T1']**2.0-forward_vol_4d**2.0)
forward_vol_3d = np.sqrt((earnings_dates_for_tickers['time_3d_T2']*earnings_dates_for_tickers['straddle_iv_3d_T2']**2.0-earnings_dates_for_tickers['time_3d_T1']*earnings_dates_for_tickers['straddle_iv_3d_T1']**2.0)/(earnings_dates_for_tickers['time_3d_T2']-earnings_dates_for_tickers['time_3d_T1']))
event_var_3d = earnings_dates_for_tickers['time_3d_T1']*(earnings_dates_for_tickers['straddle_iv_3d_T1']**2.0-forward_vol_3d**2.0)
forward_vol_2d = np.sqrt((earnings_dates_for_tickers['time_2d_T2']*earnings_dates_for_tickers['straddle_iv_2d_T2']**2.0-earnings_dates_for_tickers['time_2d_T1']*earnings_dates_for_tickers['straddle_iv_2d_T1']**2.0)/(earnings_dates_for_tickers['time_2d_T2']-earnings_dates_for_tickers['time_2d_T1']))
event_var_2d = earnings_dates_for_tickers['time_2d_T1']*(earnings_dates_for_tickers['straddle_iv_2d_T1']**2.0-forward_vol_2d**2.0)
forward_vol_1d = np.sqrt((earnings_dates_for_tickers['time_1d_T2']*earnings_dates_for_tickers['straddle_iv_1d_T2']**2.0-earnings_dates_for_tickers['time_1d_T1']*earnings_dates_for_tickers['straddle_iv_1d_T1']**2.0)/(earnings_dates_for_tickers['time_1d_T2']-earnings_dates_for_tickers['time_1d_T1']))
event_var_1d = earnings_dates_for_tickers['time_1d_T1']*(earnings_dates_for_tickers['straddle_iv_1d_T1']**2.0-forward_vol_4d**2.0)

In [34]:
length = len(event_var_0d)
results = np.zeros((length, 4))  # Prepare array to store slope, intercept, R^2, p-value

for i in range(length):
    # Check if the initial value is NaN
    if np.isnan(event_var_0d[i]):
        results[i] = [np.nan] * 4
        continue  # Skip to the next iteration
    
    # Gather data into the y array
    y = np.sqrt(np.array([event_var_0d[i], event_var_1d[i], event_var_2d[i], event_var_3d[i], event_var_4d[i], event_var_5d[i]]))
    
    # Check if there are at least 4 non-NaN values
    if np.count_nonzero(~np.isnan(y)) >= 4:
        # Filter out NaN values from both x and y
        valid_indices = ~np.isnan(y)
        x_filtered = np.array([0, 1, 2, 3, 4, 5])[valid_indices]
        y_filtered = y[valid_indices]
        
        # Perform linear regression
        regression = linregress(x_filtered, y_filtered)
        
        slope = regression.slope
        intercept = regression.intercept
        r_value = regression.rvalue
        p_value = regression.pvalue
        std_err = regression.stderr

        # Calculate R^2
        r_squared = r_value ** 2

        # Store the results
        results[i] = [slope, intercept, r_squared, p_value]
    else:
        # Not enough data to perform a reliable regression
        results[i] = [np.nan] * 4

In [35]:
results[:,0]

array([0.00295695, 0.000935  ,        nan, ...,        nan,        nan,
              nan])

In [36]:
earnings_dates_for_tickers['slope'] = results[:,0]
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['slope_decile'] = pd.qcut(earnings_dates_for_tickers['slope'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_decile = earnings_dates_for_tickers.groupby('slope_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_decile)

slope_decile
(-0.028300000000000002, -0.00395]    90.262722
(-0.00395, -0.00238]                 44.914255
(-0.00238, -0.00157]                 11.040312
(-0.00157, -0.000959]                68.201811
(-0.000959, -0.000355]                8.773444
(-0.000355, 0.000213]               -75.223889
(0.000213, 0.00084]                 -51.906668
(0.00084, 0.00161]                  -26.519052
(0.00161, 0.00302]                   -0.468634
(0.00302, 0.0313]                    79.304824
Name: straddle_pnl, dtype: float64


In [37]:
earnings_dates_for_tickers['event_var_0d/5d'] = np.sqrt(event_var_0d)/np.sqrt(event_var_5d)
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['event_var_0d/5d_decile'] = pd.qcut(earnings_dates_for_tickers['event_var_0d/5d'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_decile = earnings_dates_for_tickers.groupby('event_var_0d/5d_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_decile)

event_var_0d/5d_decile
(0.196, 0.73]      40.607850
(0.73, 0.839]      21.090807
(0.839, 0.914]    -46.166353
(0.914, 0.967]    -81.664113
(0.967, 1.015]    -20.107372
(1.015, 1.067]    -16.258767
(1.067, 1.131]      5.224265
(1.131, 1.209]     46.961187
(1.209, 1.371]     63.630680
(1.371, 18.175]    46.171802
Name: straddle_pnl, dtype: float64


In [38]:
earnings_dates_for_tickers['slope/intercept'] = results[:,0]/results[:,1]
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['slope/intercept_decile'] = pd.qcut(earnings_dates_for_tickers['slope/intercept'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_decile = earnings_dates_for_tickers.groupby('slope/intercept_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_decile)

slope/intercept_decile
(-0.225, -0.0548]      72.080649
(-0.0548, -0.0369]     75.703051
(-0.0369, -0.0249]     50.606719
(-0.0249, -0.0154]     31.092443
(-0.0154, -0.00633]   -24.396054
(-0.00633, 0.00389]   -53.272032
(0.00389, 0.0145]     -54.442556
(0.0145, 0.0293]      -23.256445
(0.0293, 0.0612]       41.571909
(0.0612, 5.058]        37.499046
Name: straddle_pnl, dtype: float64


In [39]:
earnings_dates_for_tickers['slope/event_vol_0d'] = results[:,0]/np.sqrt(event_var_0d)
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['slope/event_vol_0d_decile'] = pd.qcut(earnings_dates_for_tickers['slope/event_vol_0d'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_decile = earnings_dates_for_tickers.groupby('slope/event_vol_0d_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_decile)

slope/event_vol_0d_decile
(-0.837, -0.0553]      72.449892
(-0.0553, -0.0367]     68.307633
(-0.0367, -0.025]      53.748790
(-0.025, -0.0155]      36.558937
(-0.0155, -0.00643]   -22.627018
(-0.00643, 0.00392]   -53.961315
(0.00392, 0.0145]     -56.979594
(0.0145, 0.029]       -26.174494
(0.029, 0.0591]        39.793851
(0.0591, 0.528]        44.193808
Name: straddle_pnl, dtype: float64


In [20]:
earnings_dates_for_tickers['slope_intercept_quintile'] = pd.qcut(earnings_dates_for_tickers['slope/intercept'], 5)
earnings_dates_for_tickers['kurt25_quintile'] = pd.qcut(earnings_dates_for_tickers['kurt_25'], 5)

# Group by the quintile categories and calculate the mean of 'straddle_pnl'
cross_quantile_means = earnings_dates_for_tickers.groupby(['slope_intercept_quintile', 'kurt25_quintile'])['straddle_pnl'].mean()

# Convert the results into a 5x5 DataFrame
cross_quantile_matrix = cross_quantile_means.unstack(level=-1)
cross_quantile_matrix

kurt25_quintile,"(-27.842000000000002, -1.869]","(-1.869, -0.614]","(-0.614, 0.593]","(0.593, 2.184]","(2.184, 452.588]"
slope_intercept_quintile,,,,,
"(-0.225, -0.0369]",51.544730,131.734786,-23.793949,78.397188,106.148490
"(-0.0369, -0.0154]",-2.574032,30.507936,53.539688,65.460837,85.718878
"(-0.0154, 0.00389]",-45.608870,-28.603261,2.983397,-72.526379,-83.690604
"(0.00389, 0.0293]",-74.337932,-55.630854,-7.007747,-118.637443,98.849850
"(0.0293, 5.058]",-12.268001,43.870289,40.957255,131.646697,56.078944


In [21]:
grouped = earnings_dates_for_tickers.groupby(['slope_intercept_quintile', 'kurt25_quintile'])['straddle_pnl']
mean_std_ratio = grouped.mean() / grouped.std()

# Convert the results into a 5x5 DataFrame
mean_std_ratio_matrix = mean_std_ratio.unstack(level=-1)

In [22]:
mean_std_ratio_matrix

kurt25_quintile,"(-27.842000000000002, -1.869]","(-1.869, -0.614]","(-0.614, 0.593]","(0.593, 2.184]","(2.184, 452.588]"
slope_intercept_quintile,,,,,
"(-0.225, -0.0369]",0.085542,0.236351,-0.030334,0.159737,0.235059
"(-0.0369, -0.0154]",-0.003841,0.052763,0.084942,0.111922,0.157910
"(-0.0154, 0.00389]",-0.073779,-0.045998,0.003999,-0.112194,-0.120020
"(0.00389, 0.0293]",-0.103025,-0.076796,-0.011100,-0.133409,0.217628
"(0.0293, 5.058]",-0.021765,0.082693,0.080436,0.277052,0.094157


In [27]:
np.nanmean(np.sqrt(event_var_0d)/np.sqrt(event_var_5d))

1.072537728652541

In [8]:
earnings_dates_for_tickers['rat_5dvar_0dvar'] = np.sqrt(event_var_0d)/np.sqrt(event_var_5d)
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['rat_5dvar_0dvar_decile'] = pd.qcut(earnings_dates_for_tickers['rat_5dvar_0dvar'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_decile = earnings_dates_for_tickers.groupby('rat_5dvar_0dvar_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_decile)

rat_5dvar_0dvar_decile
(0.196, 0.73]      40.607850
(0.73, 0.839]      21.090807
(0.839, 0.914]    -46.166353
(0.914, 0.967]    -81.664113
(0.967, 1.015]    -20.107372
(1.015, 1.067]    -16.258767
(1.067, 1.131]      5.224265
(1.131, 1.209]     46.961187
(1.209, 1.371]     63.630680
(1.371, 18.175]    46.171802
Name: straddle_pnl, dtype: float64


In [137]:
earnings_dates_for_tickers['event_0dvar'] = np.sqrt(event_var_0d)
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['event_0dvar_decile'] = pd.qcut(earnings_dates_for_tickers['event_0dvar'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_decile = earnings_dates_for_tickers.groupby('event_0dvar_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_decile)

event_0dvar_decile
(0.00508, 0.033]   -10.334403
(0.033, 0.039]      -4.919170
(0.039, 0.0452]    -33.711344
(0.0452, 0.0508]    54.738200
(0.0508, 0.0582]     2.805874
(0.0582, 0.0665]    40.523718
(0.0665, 0.0775]     5.745473
(0.0775, 0.0931]    44.837929
(0.0931, 0.118]     18.649894
(0.118, 0.281]      26.029763
Name: straddle_pnl, dtype: float64


: 

In [135]:
earnings_dates_for_tickers['rat_5dvar_0dvar'].count()

4517

In [66]:
np.nanmean(np.sqrt(event_var_0d))

0.06820326388689486

In [90]:
hundred[~np.isnan(hundred['straddle_iv_T1'])]

,Unnamed: 0.1,Unnamed: 0,index,Short_ticker,Name,Ticker,Date,Event Type,Adjusted Trading Date,trade_entry,...,straddle_iv_2d_T2,straddle_iv_3d_T2,straddle_iv_4d_T2,straddle_iv_5d_T2,time_T2,time_1d_T2,time_2d_T2,time_3d_T2,time_4d_T2,time_5d_T2


In [ ]:
options_dict[2017].columns

Index(['secid', 'symbol', 'cp_flag', 'expiry_indicator', 'volume',
       'strike_price', 'exdate', 'open_interest', 'impl_volatility',
       'best_bid', 'best_offer', 'delta', 'gamma', 'vega', 'theta'],
      dtype='object')

In [ ]:
def straddle_entry_mid(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return np.nan
    try:
        options = options_dict[row['year_entry']].loc[(row['trade_entry'],row['Short_ticker'])]
    except:
        return np.nan
    options['strike_price'] *= 0.001

    # Calculate the absolute difference between 'strike_price' and row['stock_price_entry']
    options['abs_difference'] = abs(options['strike_price'] - row['stock_price_entry'])

    # Filter the DataFrame for rows with the minimum absolute difference
    min_abs_difference = options['abs_difference'].min()
    filtered_options = options[options['abs_difference'] == min_abs_difference]
    if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
        return (filtered_options['best_bid'].mean()+filtered_options['best_offer'].mean())/2.0
    return np.nan
earnings_dates_for_tickers['straddle_entry_mid'] = earnings_dates_for_tickers.apply(straddle_entry_mid,axis = 1)

In [ ]:
def straddle_entry(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return np.nan*8
    try:
        options = options_dict[row['year_entry']].loc[(row['trade_entry'],row['Short_ticker'])]
    except:
        return np.nan*8

    # Calculate the absolute difference between 'strike_price' and row['stock_price_entry']
    options['abs_difference'] = abs(options['strike_price'] - row['stock_price_entry']*1000)

    # Filter the DataFrame for rows with the minimum absolute difference
    min_abs_difference = options['abs_difference'].min()
    filtered_options = options[options['abs_difference'] == min_abs_difference]
    if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
        return filtered_options['impl_volatility'].mean(),filtered_options['best_bid'].sum(),filtered_options['best_offer'].sum(),filtered_options['gamma'].sum(),filtered_options['vega'].sum(),filtered_options['theta'].sum(),filtered_options['delta'].sum(),filtered_options['strike_price'].mean()
    return np.nan*8
earnings_dates_for_tickers[['straddle_entry_vol','straddle_entry_bid','straddle_entry_offer','straddle_entry_gamma','straddle_entry_vega','straddle_entry_theta','straddle_entry_delta','straddle_entry_strike']] = earnings_dates_for_tickers.apply(straddle_entry,axis = 1, result_type = 'expand')

In [ ]:
earnings_dates_for_tickers['straddle_entry_mid'].count()

5566

In [ ]:
def straddle_entry_skew_kurt(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return np.nan*5
    try:
        options = options_dict[row['year_entry']].loc[(row['trade_entry'],row['Short_ticker'])]
    except:
        return np.nan*7
    IV_25c = np.nan
    IV_25p = np.nan
    IV_10c = np.nan
    IV_10p = np.nan
    IV_50c = np.nan
    # Calculate the absolute difference between 'strike_price' and row['stock_price_entry']
    options['delta25_c_diff'] = abs(options['delta'] - 0.25)
    options['delta25_p_diff'] = abs(options['delta'] + 0.25)
    options['delta10_c_diff'] = abs(options['delta'] - 0.1)
    options['delta10_p_diff'] = abs(options['delta'] + 0.1)
    options['delta50_c_diff'] = abs(options['delta'] - 0.5)
    min_delta25_c = options['delta25_c_diff'].min()
    min_delta25_p = options['delta25_p_diff'].min()
    min_delta10_c = options['delta10_c_diff'].min()
    min_delta10_p = options['delta10_p_diff'].min()
    min_delta50_c = options['delta50_c_diff'].min()
    if min_delta25_c <= 0.05:
        IV_25c = options[options['delta25_c_diff'] == min_delta25_c]['impl_volatility'].mean()
    if min_delta25_p <= 0.05:
        IV_25p = options[options['delta25_p_diff'] == min_delta25_p]['impl_volatility'].mean()
    if min_delta10_c <= 0.02:
        IV_10c = options[options['delta10_c_diff'] == min_delta10_c]['impl_volatility'].mean()
    if min_delta10_p <= 0.05:
        IV_10p = options[options['delta10_p_diff'] == min_delta10_p]['impl_volatility'].mean()
    if min_delta50_c <= 0.1:
        IV_50c = options[options['delta50_c_diff'] == min_delta50_c]['impl_volatility'].mean()
    return IV_25c,IV_25p,IV_10c,IV_10p,IV_50c
earnings_dates_for_tickers[['IV_25c','IV_25p','IV_10c','IV_10p','IV_50c']] = earnings_dates_for_tickers.apply(straddle_entry_skew_kurt,axis = 1, result_type = 'expand')

In [ ]:
def straddle_exit_mid(row):
    if row['year_entry'] < 2017 or row['year_entry'] > 2023 or row['year_exit'] < 2017 or row['year_exit'] > 2023:
        return np.nan
    try:
        options = options_dict[row['year_exit']].loc[(row['trade_exit'],row['Short_ticker'])]
    except:
        return np.nan
    options['strike_price'] *= 0.001

    # Calculate the absolute difference between 'strike_price' and row['stock_price_entry']
    options['abs_difference'] = abs(options['strike_price'] - row['stock_price_entry'])

    # Filter the DataFrame for rows with the minimum absolute difference
    min_abs_difference = options['abs_difference'].min()
    filtered_options = options[options['abs_difference'] <= min_abs_difference+0.0001]
    if len(filtered_options) ==2 and ('C' in filtered_options['cp_flag'].values) and ('P' in filtered_options['cp_flag'].values):
        return (filtered_options['best_bid'].mean()+filtered_options['best_offer'].mean())/2.0
    return np.nan
earnings_dates_for_tickers['straddle_exit_mid'] = earnings_dates_for_tickers.apply(straddle_exit_mid,axis = 1)

In [ ]:
earnings_dates_for_tickers['straddle_exit_mid'].count()

4693

In [ ]:
earnings_dates_for_tickers['straddle_pnl'] = (earnings_dates_for_tickers['straddle_entry_mid'] - earnings_dates_for_tickers['straddle_exit_mid'])*1000.0/earnings_dates_for_tickers['straddle_entry_mid']

In [ ]:
earnings_dates_for_tickers['straddle_pnl'].count()

4263

In [ ]:
earnings_dates_for_tickers['straddle_pnl'].std()

720.5208311693931

In [ ]:
earnings_dates_for_tickers.columns

Index(['Unnamed: 0', 'index', 'Short_ticker', 'Name', 'Ticker', 'Date',
       'Event Type', 'Adjusted Trading Date', 'trade_entry', 'trade_exit',
       'stock_price_entry', 'stock_price_exit', 'year_entry', 'year_exit',
       'straddle_entry_mid', 'straddle_exit_mid', 'straddle_pnl',
       'straddle_entry_bid', 'straddle_entry_offer', 'straddle_entry_gamma',
       'straddle_entry_vega', 'straddle_entry_theta', 'straddle_entry_delta',
       'straddle_entry_strike'],
      dtype='object')

In [ ]:
earnings_dates_for_tickers[np.abs(earnings_dates_for_tickers['straddle_entry_strike']*0.001-earnings_dates_for_tickers['stock_price_entry'])/earnings_dates_for_tickers['stock_price_entry'] < 0.05]['straddle_pnl'].count()

1423

In [ ]:
earnings_dates_for_tickers['skew_25'] = 4.4478*(earnings_dates_for_tickers['IV_25c']-earnings_dates_for_tickers['IV_25p'])/earnings_dates_for_tickers['IV_50c']

In [ ]:
earnings_dates_for_tickers['kurt_25'] = 52.7546*(0.5*(earnings_dates_for_tickers['IV_25c']+earnings_dates_for_tickers['IV_25p'])/earnings_dates_for_tickers['IV_50c'] - 1.0)
earnings_dates_for_tickers['kurt_25'].count()

5594

In [ ]:
earnings_dates_for_tickers['skew_25'].count()

5594

In [ ]:
(earnings_dates_for_tickers[['straddle_pnl', 'skew_25']].notna().all(axis=1)).sum()

3613

In [ ]:
(earnings_dates_for_tickers[['straddle_pnl', 'kurt_25']].notna().all(axis=1)).sum()

3613

In [9]:
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['skew_25_decile'] = pd.qcut(earnings_dates_for_tickers['skew_25'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_decile = earnings_dates_for_tickers.groupby('skew_25_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_decile)

skew_25_decile
(-20.905, -0.762]    45.359169
(-0.762, -0.586]     22.012424
(-0.586, -0.468]    -20.934392
(-0.468, -0.368]     18.731875
(-0.368, -0.272]     25.583620
(-0.272, -0.171]    -32.543577
(-0.171, -0.0677]    73.838410
(-0.0677, 0.0608]    75.767279
(0.0608, 0.263]     -20.306298
(0.263, 11.769]     -52.400244
Name: straddle_pnl, dtype: float64


In [ ]:
earnings_dates_for_tickers[earnings_dates_for_tickers['skew_25']<0.0608]['straddle_pnl'].mean()

25.82727723211073

In [1]:
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['kurt_25_decile'] = pd.qcut(earnings_dates_for_tickers['kurt_25'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_kurt_decile = earnings_dates_for_tickers.groupby('kurt_25_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_kurt_decile)

NameError: name 'pd' is not defined

In [ ]:
num_quantiles = 10

# Create deciles based on 'skew_25'
earnings_dates_for_tickers['straddle_vol_decile'] = pd.qcut(earnings_dates_for_tickers['straddle_entry_vol'], num_quantiles)

# Calculate the mean 'straddle_pnl' for each decile
mean_straddle_pnl_by_vol_decile = earnings_dates_for_tickers.groupby('straddle_vol_decile')['straddle_pnl'].mean()

print(mean_straddle_pnl_by_vol_decile)

straddle_vol_decile
(0.206, 0.501]   -69.254183
(0.501, 0.621]     0.756881
(0.621, 0.73]     23.058441
(0.73, 0.849]     17.395843
(0.849, 0.952]    43.000185
(0.952, 1.104]    45.939242
(1.104, 1.293]   -27.507660
(1.293, 1.566]   -41.562691
(1.566, 1.957]     2.208024
(1.957, 2.982]    67.354911
Name: straddle_pnl, dtype: float64


In [ ]:
earnings_dates_for_tickers[earnings_dates_for_tickers['kurt_25']>=1.279]['straddle_pnl'].mean()

45.867461955398504

In [ ]:
earnings_dates_for_tickers['skew_25_quartile'] = pd.qcut(earnings_dates_for_tickers['skew_25'], 4)
earnings_dates_for_tickers['kurt_25_quartile'] = pd.qcut(earnings_dates_for_tickers['skew_25'], 4)



In [ ]:
earnings_dates_for_tickers.to_csv("results.csv")

In [ ]:
earnings_dates_for_tickers = pd.read_csv("results.csv")